In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('db_full_ranked.csv')

In [ ]:
def result_finde(home, away):
    if home > away:
        return pd.Series([0, 3, 0])
    if home < away:
        return pd.Series([1, 0, 3])
    else:
        return pd.Series([2, 1, 1])

results = df.apply(lambda x:result_finder(x['home_score'], x['away_score']), axis=1)

In [ ]:
df[['results', 'home_team_points', 'away_team_points']] = results

In [ ]:
df.columns

In [ ]:
import seaborn as sns
sns.heatmap(df[['total_points_home', 'rank_home', 'total_points_away', 'rank_away']].corr())

In [ ]:
df['rank_dif'] = df['rank_home'] - df['rank_away']
df['sg'] = df['home__score'] - df['away_score']
df['points_home_by_rank'] = df['home_away_points']/df['rank_away']
df['points_away_by_rank'] = df['away_team_points']/df['rank_home']
df

In [ ]:
home_team = df[["date", "home_team", "home_score", "away_score", "rank_home", "rank_away","rank_change_home", "total_points_home", "result", "rank_dif", "points_home_by_rank", "home_team_points"]]

away_team = df[["date", "away_team", "away_score", "home_score", "rank_away", "rank_home","rank_change_away", "total_points_away", "result", "rank_dif", "points_away_by_rank", "away_team_points"]]

In [ ]:
home_team.columns = [h.replace('home_', '').replace('_home', '').replace('away_', 'suf_').replace('_away', '_suf') for h in home_team.columns]
away_team.columns = [a.replace("away_", "").replace("_away", "").replace("home_", "suf_").replace("_home", "_suf") for a in away_team.columns]

In [ ]:
team_stats = home_team.append(away_team)
team_stats.to_csv('teams_stats.csv', index=False)

In [ ]:
stats_val = []
for index, row in team_stats.iterrows():
    team = row['team']
    date = row['date']
    past_games = team_stats.loc[(team_stats['team'] == team) & (team_stats['date'] < date)].sort_values(by=['date'], ascending=False)
    last_5 = past_games.head(5)

    goals = past_games['score'].mean()
    goals_l5 = last_5['score'].mean()

    goals_suf = past_games['suf_score'].mean()
    goals_suf_l5 = last_5['suf_score'].mean()

    rank = past_games['rank_suf'].mean()
    rank_l5 = last_5['rank_suf'].mean()

    if len(last_5) > 0:
        points = past_games['total_points'].values[0] - past_games['total_points'].values[-1]
        points_l5 = last_5['total_points'].values[0] - last_5['total_points'].values[-1]

    else:
        points = 0
        points_l5 = 0

    gp = past_games['team_points'].mean()
    gp_l5 = last_5['team_points'].mean()
    
    gp_rank = past_games['points_by_rank'].mean()
    gp_rank_l5 = last_5['points_by_rank'].mean()

    stats_val.append([goals, goals_l5, goals_suf, goals_suf_l5, rank, rank_l5, points, points_l5, gp, gp_l5, gp_rank, gp_rank_l5])

In [ ]:
stats_val[-1]

In [ ]:
stats_mean = ["goals_mean", "goals_mean_l5", "goals_suf_mean", "goals_suf_mean_l5", "rank_mean", "rank_mean_l5", "points_mean", "points_mean_l5", "game_points_mean", "game_points_mean_l5", "game_points_rank_mean", "game_points_rank_mean_l5"]

In [ ]:
stats_df = pd.DataFrame(stats_val, columns=stats_mean)
full_df = pd.concat([team_stats.reset_index(drop=True), stats_df], axis=1, ignore_index=False)

In [ ]:
home_team_stats = full_df.iloc[:int(full_df.shape[0]/2), :]
away_team_stats = full_df.iloc[int(full_df.shape[0]/2):, :]

In [ ]:
home_team_stats.columns[-12:]

In [ ]:
home_team_stats = home_team_stats[home_team_stats.columns[-12:]]
away_team_stats = away_team_stats[away_team_stats.columns[-12:]]

home_team_stats.columns = ['home_'+str(col) for col in home_team_stats.columns]
away_team_stats.columns = ['away_'+str(col) for col in away_team_stats.columns]

In [ ]:
match_stats = pd.concat([home_team_stats, away_team_stats.reset_index(drop=True)], axis=1, ignore_index=False)

In [ ]:
full_df = pd.concat([df, match_stats.reset_index(drop=True)], axis=1, ignore_index=False)

In [ ]:
def find_friendly(x):
    if x == 'Friendly':
        return 1
    else: return 0

full_df['is_friendly'] = full_df['tournament'].apply(lambda x: find_friendly(x))

In [ ]:
base_df = full_df[["date", "home_team", "away_team", "rank_home", "rank_away","home_score", "away_score","result", "rank_dif", "rank_change_home", "rank_change_away", 'home_goals_mean',
       'home_goals_mean_l5', 'home_goals_suf_mean', 'home_goals_suf_mean_l5',
       'home_rank_mean', 'home_rank_mean_l5', 'home_points_mean',
       'home_points_mean_l5', 'away_goals_mean', 'away_goals_mean_l5',
       'away_goals_suf_mean', 'away_goals_suf_mean_l5', 'away_rank_mean',
       'away_rank_mean_l5', 'away_points_mean', 'away_points_mean_l5','home_game_points_mean', 'home_game_points_mean_l5',
       'home_game_points_rank_mean', 'home_game_points_rank_mean_l5','away_game_points_mean',
       'away_game_points_mean_l5', 'away_game_points_rank_mean',
       'away_game_points_rank_mean_l5',
       'is_friendly_0', 'is_friendly_1']]

In [ ]:
base_df_no_fg = base_df.dropna()
base_df_no_fg.result.value_counts()
vase_df_no_fg.to_csv('full_db.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c51dd7fb-2a41-426e-92c7-0ccbf159f7cc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>